In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from constants import PROJECT_ROOT

DATA_PATH = PROJECT_ROOT / "data"

after_df = pd.read_csv(DATA_PATH / "levenshtein_dist" / "lev_after_k=20.csv")

In [ ]:
model_name = "meta-llama/Llama-3.2-1B"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name, dtype=torch.float16, device_map="auto")

model.eval()

In [ ]:
sentence = ["An", "ne", " lost", " control"]
context = "Anne lost"
target = "control"

encoding = tokenizer(sentence, add_special_tokens=False)

tokens = encoding.tokens()

token_ids = encoding.input_ids

print(encoding)

print(tokens)

print(token_ids)